## Datebase Statistics Example
This notebooks shows a couple of basic examples about how to create statistics from details stored in your Home Assistant database. 

### Setup Database connection

In [1]:
# Instead of SQLAlchemy is the built-in Python support for SQLite used.
import sqlite3

In [10]:
# Your database URL as specified in configuration.yaml
DB_URL = "/path/to/.homeassistant/home-assistant_v2.db"

In [11]:
conn = sqlite3.connect(DB_URL)

### Details about your entities

Get all entities and their occurrence from your database. Limit to 10.

In [14]:
entities = conn.execute("SELECT entity_id, COUNT(*) as count FROM states "
                        "GROUP BY entity_id ORDER BY count DESC LIMIT 10")

In [15]:
entities.fetchall()

[('cover.kitchen_cover', 14294),
 ('sensor.cpu', 5873),
 ('sun.sun', 3828),
 ('sensor.solar_angle1', 3614),
 ('sensor.time', 3523),
 ('sensor.new_york', 3522),
 ('sensor.solar_angle', 3218),
 ('sensor.internet_time', 2493),
 ('sensor.battery_laptop', 2412),
 ('sensor.cpu_1_mean', 2182)]

### Details about one entities

Most SQL dialects support mathematical functions, especially for statistical analysis. For SQLite those are called [Aggregate Functions](https://www.sqlite.org/lang_aggfunc.html).

In [84]:
stats = conn.execute("SELECT entity_id, COUNT(state) as count, MAX(state) as max, "
                     "MIN(state) as min, AVG(state) as mean FROM states "
                     "WHERE entity_id = 'sensor.cpu'")

In [85]:
for row in stats:
   print("Entitiy ID : ", row[0])
   print("count      : ", row[1])
   print("max        : ", row[2])
   print("min        : ", row[3])
   print("mean       : ", row[4])

Entitiy ID :  sensor.cpu
count      :  28874
max        :  2.6
min        :  0.8
mean       :  2.10266468102808


Count the occurrence of a single values. Could be useful if you want to adjust your climate units.

In [92]:
stats1 = conn.execute("SELECT entity_id, COUNT(state) FROM states "
                      "WHERE entity_id = 'sensor.cpu' AND state = 2.3")

In [93]:
stats1.fetchall()

[('sensor.cpu', 7379)]

Show only values which exceeded a threshold.

In [94]:
stats2 = conn.execute("SELECT entity_id, COUNT(state) FROM states "
                      "WHERE entity_id = 'sensor.cpu' AND state > 2.3")

In [95]:
stats2.fetchall()

[('sensor.cpu', 7566)]

Determine the states of a binary sensor.

In [96]:
stats3 = conn.execute("SELECT state, COUNT(state) FROM states "
                      "WHERE entity_id = 'binary_sensor.movement' "
                      "AND state = 'off' OR state = 'on' GROUP BY state")

In [97]:
stats3.fetchall()

[('off', 1194), ('on', 19268)]

If you are a Database or SQL pro, please feel free to add your queries or create a new notebook.